# How to pass runtime secrets to runnables

To pass runtime secrets to LangChain runnables using RunnableConfig, you can follow the approach demonstrated in the example you provided. This allows you to securely pass sensitive information (such as API keys or passwords) to tools or runnables at runtime without exposing them in traces or logs.

# Key Steps:

Use RunnableConfig: You will use RunnableConfig to pass the secrets at runtime. This ensures that the secrets are not traced or logged in the LangSmith trace.

Prefix Secrets with __: To ensure secrets are not traced, you need to pass them with a __ prefix. This is how LangChain handles secrets securely.

Invoke the Runnable: When you invoke the tool or runnable, you can pass the secrets as part of the configurable field in RunnableConfig.

In [1]:
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool

# Define a tool that uses a secret passed via RunnableConfig
@tool
def foo(x: int, config: RunnableConfig) -> int:
    """Sum x and a secret int."""
    # Access the secret using the 'configurable' field with the '__' prefix
    secret_int = config["configurable"]["__top_secret_int"]
    return x + secret_int

# Example runtime secret to pass along
runtime_secrets = {
    "configurable": {
        "__top_secret_int": 2,  # Secret value prefixed with __
        "traced_key": "bar"  # Regular field that will be traced
    }
}

# Invoke the tool with the secret and non-secret config
result = foo.invoke({"x": 5}, runtime_secrets)

print(f"Result: {result}")


Result: 7


# Explanation:

foo Tool: The foo function takes an integer x and a config object, which is an instance of RunnableConfig. The secret value __top_secret_int is passed within the config dictionary.

Secrets Handling: Secrets are passed with the __ prefix (e.g., __top_secret_int), which tells LangChain to handle them securely and not include them in the trace logs.

Invocation: When invoking foo, you pass the runtime_secrets dictionary containing the secrets and other non-sensitive configuration.

# Output:
The result of the invocation would be x + 2, since the secret top_secret_int is 2 and x is passed as 5, so the result will be 7.

LangSmith will record the non-secret key (traced_key), but __top_secret_int will not be logged as part of the trace for security.

# Additional Considerations:
Secrets Handling: Always ensure that sensitive information like API keys, passwords, and tokens are passed with the __ prefix, as this will keep them secure and not logged in trace data.

Tool Configuration: You can configure multiple secrets or other non-sensitive configuration options using RunnableConfig, making it flexible to pass additional parameters securely